<a href="https://colab.research.google.com/github/muhaiminsk/vr-activity-classification/blob/main/Demo_VR_Activity_classf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install keras
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


In [2]:
from google.colab import files
import io
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
from datetime import datetime
import kerastuner as kt

# Check GPU availability
gpus = tf.config.list_physical_devices('GPU')
print("GPU Available:", bool(gpus))

# Load and preprocess data
def load_data(uploaded_file):
    df = pd.read_csv(io.BytesIO(uploaded_file))
    required_columns = ['Time', 'No. of Packets', 'No. of Bits',
                        'Avg. Packet Length (Bytes)', 'Avg. Inter-packet Arrival Time', 'Activity']
    df = df[required_columns]
    return df

# Load training data
print("Upload training dataset:")
uploaded_train = files.upload()
df_train = load_data(list(uploaded_train.values())[0])

# Preprocessing
le = LabelEncoder()
df_train['Activity'] = le.fit_transform(df_train['Activity'])
X_train = df_train.drop('Activity', axis=1)
y_train = df_train['Activity']

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Apply SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train_scaled, y_train)

# Load test data
print("\nUpload test dataset:")
uploaded_test = files.upload()
df_test = load_data(list(uploaded_test.values())[0])
df_test['Activity'] = le.transform(df_test['Activity'])
X_test = df_test.drop('Activity', axis=1)
y_test = df_test['Activity']
X_test_scaled = scaler.transform(X_test)

# Model Building
def model_builder(hp):
    model = Sequential()
    model.add(Dense(
        units=hp.Int('units', 64, 256, step=64),
        activation='relu',
        input_shape=(X_res.shape[1],)
    ))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(le.classes_), activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('lr', [1e-3, 5e-4, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Hyperparameter Tuning
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=50,
    directory='tuning',
    project_name='activity_recognition'
)

# Search and train
tuner.search(X_res, y_res, epochs=50, validation_split=0.2, batch_size=32)

# Best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluation
test_loss, test_acc = best_model.evaluate(X_test_scaled, y_test)
print(f"\nTest Accuracy: {test_acc*100:.2f}%")

# Save predictions
df_test['Predicted'] = le.inverse_transform(best_model.predict(X_test_scaled).argmax(axis=1))
df_test.to_csv('predictions.csv', index=False)
files.download('predictions.csv')

Trial 12 Complete [00h 00m 08s]
val_accuracy: 0.31529316306114197

Best val_accuracy So Far: 0.5633456110954285
Total elapsed time: 00h 01m 22s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.8720 - loss: 0.3603

Test Accuracy: 79.65%
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>